In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.dates as mtd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import confusion_matrix, classification_report, plot_roc_curve,accuracy_score
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier  
from sklearn.linear_model import LogisticRegression 
from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import platform, struct
print(platform.architecture()[0])
print(struct.calcsize("P") * 8)

64bit
64


In [ ]:
flightdataframeJan = pd.read_csv('/content/drive/MyDrive/AbiML/Internship Tasks/internshipDatas/flightdataJanuary.csv')
flightdataframeFeb = pd.read_csv('/content/drive/MyDrive/AbiML/Internship Tasks/internshipDatas/flightdataFebruary.csv')
flightdataframeMar = pd.read_csv('/content/drive/MyDrive/AbiML/Internship Tasks/internshipDatas/flightdataMarch.csv')
flightdataframeApr = pd.read_csv('/content/drive/MyDrive/AbiML/Internship Tasks/internshipDatas/flightdataAprilcsv.txt')
flightdataframeMay = pd.read_csv('/content/drive/MyDrive/AbiML/Internship Tasks/internshipDatas/flightdataMay.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,13,57) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
consolidatedfivemonthsdf = pd.concat([flightdataframeJan,flightdataframeFeb,flightdataframeMar,flightdataframeApr,flightdataframeMay], axis=0)
consolidatedfivemonthsdf1=consolidatedfivemonthsdf.drop(columns=['Unnamed: 74'] ,axis=1)

In [ ]:
#consolidatedfivemonthsdf1['MONTH'].value_counts()

In [ ]:
consolidatedSelectedColumns=consolidatedfivemonthsdf1[['MONTH','DAY_OF_MONTH', 'DAY_OF_WEEK','TAIL_NUM','OP_CARRIER','ORIGIN','DEP_DEL15', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY','SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']]
#consolidatedSelectedColumns.head()

In [ ]:
consolidatedSelectedColumns.dropna(subset=['DEP_DEL15'],inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


POPULATION DATA SUMMARY 

In [ ]:
pd.DataFrame({'unique_values':consolidatedSelectedColumns.nunique(),
              'missing_values': consolidatedSelectedColumns.isna().sum(),
              'datatype':consolidatedSelectedColumns.dtypes,
             
'count':consolidatedSelectedColumns.count()
              })

,unique_values,missing_values,datatype,count
MONTH,5,0,int64,2156140
DAY_OF_MONTH,31,0,int64,2156140
DAY_OF_WEEK,7,0,int64,2156140
TAIL_NUM,5400,0,object,2156140
OP_CARRIER,22,0,object,2156140
ORIGIN,375,0,object,2156140
DEP_DEL15,2,0,float64,2156140
CARRIER_DELAY,1137,1904965,float64,251175
WEATHER_DELAY,821,1904965,float64,251175
NAS_DELAY,502,1904965,float64,251175


In [ ]:
#pd.set_option('precision', 3)
#suppress scientific Exponential format and precision 1
pd.options.display.float_format = '{:.1f}'.format
month_all_pop=consolidatedSelectedColumns.agg({'CARRIER_DELAY':['mean',lambda count: count.ne(0).sum()],'WEATHER_DELAY':['mean',lambda count: count.ne(0).sum()],'NAS_DELAY':['mean',lambda count: count.ne(0).sum()],'LATE_AIRCRAFT_DELAY':['mean',lambda count: count.ne(0).sum()],'SECURITY_DELAY':['mean',lambda count: count.ne(0).sum()]}).rename(columns={'<lambda>': 'Count'})
month_all_pop

,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,LATE_AIRCRAFT_DELAY,SECURITY_DELAY
mean,26.3,6.2,13.5,18.6,0.3
<lambda>,2038164.0,1924288.0,2027949.0,1992236.0,1907098.0


# CHOSING SAMPLES WITH systematic_sampling TECHNIQUE


In [ ]:
# Define systematic sampling function
def systematic_sampling(consolidatedSelectedColumns, step):
 
    indexes = np.arange(0, len(consolidatedSelectedColumns), step=step)
    systematic_sample = consolidatedSelectedColumns.iloc[indexes]
    return systematic_sample
 
 
# Obtain a systematic sample and save it in a new variable
systematic_sample = systematic_sampling(consolidatedSelectedColumns, 10)
 
# View sampled data frame
#display(systematic_sample)

In [ ]:
meansample= systematic_sample.agg({'CARRIER_DELAY':['mean',lambda count: count.ne(0).sum()],'WEATHER_DELAY':['mean',lambda count: count.ne(0).sum()],'NAS_DELAY':['mean',lambda count: count.ne(0).sum()],'LATE_AIRCRAFT_DELAY':['mean',lambda count: count.ne(0).sum()],'SECURITY_DELAY':['mean',lambda count: count.ne(0).sum()]}).rename(columns={'<lambda_0>': 'Count'})
meansample

,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,LATE_AIRCRAFT_DELAY,SECURITY_DELAY
mean,26.370065,6.103821,13.463552,18.56571,0.322669
<lambda>,203834.000000,192325.000000,202622.000000,199193.00000,190580.000000


### Sample Summary

In [ ]:
pd.DataFrame({'unique_values':systematic_sample.nunique(),
              'missing_values': systematic_sample.isna().sum(),
              'datatype':systematic_sample.dtypes,
              'count':systematic_sample.count()
              })

,unique_values,missing_values,datatype,count
MONTH,5,0,int64,215614
DAY_OF_MONTH,31,0,int64,215614
DAY_OF_WEEK,7,0,int64,215614
TAIL_NUM,5364,0,object,215614
OP_CARRIER,22,0,object,215614
ORIGIN,373,0,object,215614
DEP_DEL15,2,0,float64,215614
CARRIER_DELAY,494,190359,float64,25255
WEATHER_DELAY,307,190359,float64,25255
NAS_DELAY,277,190359,float64,25255


### Inorder to test whether our samples collected from population are good representation of population need Statistical testing on Categorial columns and Numerical columns between populatipon and samples
* Numerical variables must be checked with the Kolmogorov-Smirnov test
* Categorical variables must be checked with the Pearson’s chi-square test.

In [ ]:
categoricalDFsample=systematic_sample[['MONTH',	'DAY_OF_MONTH','DAY_OF_WEEK','TAIL_NUM','OP_CARRIER',	'ORIGIN']]	
numericDFsample=systematic_sample[['CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY',	'SECURITY_DELAY',	'LATE_AIRCRAFT_DELAY']]

In [ ]:
categoricalDFpopulation=consolidatedSelectedColumns[['MONTH',	'DAY_OF_MONTH','DAY_OF_WEEK','TAIL_NUM','OP_CARRIER',	'ORIGIN']]	
numericDFpopulation=consolidatedSelectedColumns[['CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY',	'SECURITY_DELAY',	'LATE_AIRCRAFT_DELAY']]

# Kolmogorov-Smirnov test

In [ ]:
from numpy.random import seed
from numpy.random import randint
from scipy.stats import ks_2samp
for col in numericDFpopulation: 
  value, pvalue = ks_2samp(numericDFsample[col], numericDFpopulation[col])
  print(value, pvalue)
  if pvalue > 0.05:
    print(numericDFsample[col].name,'vs', numericDFpopulation[col].name,'Samples are sample is statistically significant as population (fail to reject H0)')
  else:
    print(numericDFsample[col].name,'vs', numericDFpopulation[col].name,'Samples are  statistically NOT significant as population (reject H0)')

0.0006831652861131465 0.9999884747234531
CARRIER_DELAY vs CARRIER_DELAY Samples are sample is statistically significant as population (fail to reject H0)
0.0007040359160351284 0.9999754147364329
WEATHER_DELAY vs WEATHER_DELAY Samples are sample is statistically significant as population (fail to reject H0)
0.0009772092721251902 0.9920469856407877
NAS_DELAY vs NAS_DELAY Samples are sample is statistically significant as population (fail to reject H0)
0.0006817739107850135 0.9999890703265211
SECURITY_DELAY vs SECURITY_DELAY Samples are sample is statistically significant as population (fail to reject H0)
0.0007258341295092174 0.9999494534436161
LATE_AIRCRAFT_DELAY vs LATE_AIRCRAFT_DELAY Samples are sample is statistically significant as population (fail to reject H0)


# Pearson’s chi-square test

In [ ]:
from numpy.random import seed
from numpy.random import randint
from scipy.stats import ks_2samp
for col in categoricalDFpopulation: 
  value, pvalue = ks_2samp(categoricalDFsample[col], categoricalDFpopulation[col])
  print(value, pvalue)
  if pvalue > 0.05:
    print(categoricalDFsample[col].name,'vs', categoricalDFpopulation[col].name,'Samples are  statistically significant as population (fail to reject H0)')
  else:
    print(categoricalDFsample[col].name,'vs', categoricalDFpopulation[col].name,'Samples are  statistically NOT significant as population (reject H0)')

3.2465424323335768e-06 1.0
MONTH vs MONTH Samples are  statistically significant as population (fail to reject H0)
0.0011924086562097036 0.9432386472403238
DAY_OF_MONTH vs DAY_OF_MONTH Samples are  statistically significant as population (fail to reject H0)
0.0009280473438644798 0.9959106818183939
DAY_OF_WEEK vs DAY_OF_WEEK Samples are  statistically significant as population (fail to reject H0)
0.0014108545827265229 0.8300859891121797
TAIL_NUM vs TAIL_NUM Samples are  statistically significant as population (fail to reject H0)
4.591538582837673e-05 1.0
OP_CARRIER vs OP_CARRIER Samples are  statistically significant as population (fail to reject H0)
0.001175248360496095 0.9494412014433004
ORIGIN vs ORIGIN Samples are  statistically significant as population (fail to reject H0)


Both test, we failed to reject null hypothesis, (we can say samples taken are good representation of population) 

# Weight calculation on all continouse variable columns

## Tailnum vs DEP_DEL count weight

In [ ]:
tail_dep_del = systematic_sample[['TAIL_NUM','DEP_DEL15']].groupby('TAIL_NUM').sum().sort_values(by='DEP_DEL15',ascending=False).reset_index()
tail_dep_del.rename({'DEP_DEL15':'TAIL_NUM_DELAY_COUNT'},inplace=True, axis=1)
tail_dep_del['TAIL_DELAY_weight'] = pd.qcut(tail_dep_del['TAIL_NUM_DELAY_COUNT'], 60, labels = False,duplicates='drop')
#tail_dep_del
systematic_sample1 = systematic_sample.merge(tail_dep_del, left_on='TAIL_NUM', right_on='TAIL_NUM')
systematic_sample1.drop(columns=['TAIL_NUM_DELAY_COUNT'], inplace=True, axis=1)
#systematic_sample1

In [ ]:
tail_dep_del_carrier = systematic_sample1[['TAIL_NUM','CARRIER_DELAY']].groupby('TAIL_NUM').sum().sort_values(by='CARRIER_DELAY',ascending=False).reset_index()
tail_dep_del_carrier['TAIL_DELAY_CARRIER_weight'] = pd.qcut(tail_dep_del_carrier['CARRIER_DELAY'], 120, labels = False,duplicates='drop')
print(tail_dep_del_carrier)
systematic_sample2 = systematic_sample1.merge(tail_dep_del_carrier, left_on='TAIL_NUM', right_on='TAIL_NUM')
systematic_sample2.drop(columns=['CARRIER_DELAY_x','CARRIER_DELAY_y'], inplace=True, axis=1)

     TAIL_NUM  CARRIER_DELAY  TAIL_DELAY_CARRIER_weight
0      N946SW         2223.0                         99
1      N951SW         2064.0                         99
2      N746SK         1835.0                         99
3       239NV         1827.0                         99
4      N502AE         1753.0                         99
...       ...            ...                        ...
5359   N569AS            0.0                          0
5360   N397DA            0.0                          0
5361   N858DZ            0.0                          0
5362   N858AE            0.0                          0
5363   N683NK            0.0                          0

[5364 rows x 3 columns]


In [ ]:
#systematic_sample2

In [ ]:
tail_dep_del_security = systematic_sample2[['TAIL_NUM','SECURITY_DELAY']].groupby('TAIL_NUM').sum().sort_values(by='SECURITY_DELAY',ascending=False).reset_index()
tail_dep_del_security['TAIL_DELAY_SECURITY_weight'] = pd.qcut(tail_dep_del_security['SECURITY_DELAY'], 50, labels = False,duplicates='drop')
print(tail_dep_del_security)
systematic_sample3 = systematic_sample2.merge(tail_dep_del_security, left_on='TAIL_NUM', right_on='TAIL_NUM')
systematic_sample3.drop(columns=['SECURITY_DELAY_x','SECURITY_DELAY_y'], inplace=True, axis=1)

     TAIL_NUM  SECURITY_DELAY  TAIL_DELAY_SECURITY_weight
0      N632SK           691.0                           1
1      N150UW           474.0                           1
2      N616NK           369.0                           1
3      N626NK           292.0                           1
4      N670NK           251.0                           1
...       ...             ...                         ...
5359   N404YX             0.0                           0
5360   N404WN             0.0                           0
5361   N404UA             0.0                           0
5362   N404SY             0.0                           0
5363   N999JQ             0.0                           0

[5364 rows x 3 columns]


In [ ]:
#systematic_sample3

In [ ]:
tail_dep_del_weather = systematic_sample3[['TAIL_NUM','WEATHER_DELAY']].groupby('TAIL_NUM').sum().sort_values(by='WEATHER_DELAY',ascending=False).reset_index()
tail_dep_del_weather['TAIL_DELAY_WEATHER_weight'] = pd.qcut(tail_dep_del_weather['WEATHER_DELAY'], 80, labels = False,duplicates='drop')
print(tail_dep_del_weather)
systematic_sample4 = systematic_sample3.merge(tail_dep_del_weather, left_on='TAIL_NUM', right_on='TAIL_NUM')
systematic_sample4.drop(columns=['WEATHER_DELAY_x','WEATHER_DELAY_y'], inplace=True, axis=1)

     TAIL_NUM  WEATHER_DELAY  TAIL_DELAY_WEATHER_weight
0      N8688C         1418.0                         21
1      N45440         1333.0                         21
2      N435SW         1259.0                         21
3      N9018E         1227.0                         21
4      N919SW         1206.0                         21
...       ...            ...                        ...
5359   N449YX            0.0                          0
5360   N450WN            0.0                          0
5361   N450YX            0.0                          0
5362   N451AW            0.0                          0
5363   N999JQ            0.0                          0

[5364 rows x 3 columns]


In [ ]:
tail_dep_del_NAS = systematic_sample4[['TAIL_NUM','NAS_DELAY']].groupby('TAIL_NUM').sum().sort_values(by='NAS_DELAY',ascending=False).reset_index()
tail_dep_del_NAS['TAIL_DELAY_NAS_weight'] = pd.qcut(tail_dep_del_NAS['NAS_DELAY'], 50, labels = False,duplicates='drop')
print(tail_dep_del_NAS)
systematic_sample5 = systematic_sample4.merge(tail_dep_del_NAS, left_on='TAIL_NUM', right_on='TAIL_NUM')
systematic_sample5.drop(columns=['NAS_DELAY_x','NAS_DELAY_y'], inplace=True, axis=1)

     TAIL_NUM  NAS_DELAY  TAIL_DELAY_NAS_weight
0      N641RW     1011.0                     38
1      N933NK      981.0                     38
2      N9011P      966.0                     38
3      N662NK      964.0                     38
4      N84378      907.0                     38
...       ...        ...                    ...
5359   N224AK        0.0                      0
5360   N607SK        0.0                      0
5361   N2250U        0.0                      0
5362   N466UA        0.0                      0
5363   N813UA        0.0                      0

[5364 rows x 3 columns]


In [ ]:
tail_dep_del_lateair = systematic_sample5[['TAIL_NUM','LATE_AIRCRAFT_DELAY']].groupby('TAIL_NUM').sum().sort_values(by='LATE_AIRCRAFT_DELAY',ascending=False).reset_index()
tail_dep_del_lateair['TAIL_DELAY_LATE_AIRCRAFT_weight'] = pd.qcut(tail_dep_del_lateair['LATE_AIRCRAFT_DELAY'], 90, labels = False,duplicates='drop')
print(tail_dep_del_lateair)
systematic_sample6 = systematic_sample5.merge(tail_dep_del_lateair, left_on='TAIL_NUM', right_on='TAIL_NUM')
systematic_sample6.drop(columns=['LATE_AIRCRAFT_DELAY_x','LATE_AIRCRAFT_DELAY_y'], inplace=True, axis=1)

     TAIL_NUM  LATE_AIRCRAFT_DELAY  TAIL_DELAY_LATE_AIRCRAFT_weight
0      N908FJ               1737.0                               59
1      N732SK               1502.0                               59
2      N189UW               1467.0                               59
3      N998AN               1286.0                               59
4      N934FJ               1184.0                               59
...       ...                  ...                              ...
5359   N837AW                  0.0                                0
5360   N837AN                  0.0                                0
5361   N379FR                  0.0                                0
5362   N14177                  0.0                                0
5363   N387DA                  0.0                                0

[5364 rows x 3 columns]


In [ ]:
#systematic_sample6

In [ ]:
systematic_sample6.drop(columns=['TAIL_NUM'], inplace=True, axis=1)
systematic_sample6

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER,ORIGIN,DEP_DEL15,TAIL_DELAY_weight,TAIL_DELAY_CARRIER_weight,TAIL_DELAY_SECURITY_weight,TAIL_DELAY_WEATHER_weight,TAIL_DELAY_NAS_weight,TAIL_DELAY_LATE_AIRCRAFT_weight
0,1,17,7,DL,DFW,0.0,3,58,0,0,15,0
1,1,18,1,DL,ATL,0.0,3,58,0,0,15,0
2,1,20,3,DL,DTW,0.0,3,58,0,0,15,0
3,1,2,6,DL,FAR,0.0,3,58,0,0,15,0
4,1,4,1,DL,DCA,0.0,3,58,0,0,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...
215609,5,7,5,G7,MBS,0.0,0,23,0,0,0,0
215610,5,6,4,G7,ORD,0.0,0,23,0,0,0,0
215611,5,28,5,C5,SYR,0.0,0,0,0,0,0,0
215612,5,26,3,C5,IAD,0.0,0,0,0,0,0,0


In [ ]:
#df[['A', 'C']] = df[['A', 'C']].apply(pd.to_numeric)
print(systematic_sample6.dtypes)

MONTH                                int64
DAY_OF_MONTH                         int64
DAY_OF_WEEK                          int64
OP_CARRIER                          object
ORIGIN                              object
DEP_DEL15                          float64
TAIL_DELAY_weight                    int64
TAIL_DELAY_CARRIER_weight            int64
TAIL_DELAY_SECURITY_weight           int64
TAIL_DELAY_WEATHER_weight            int64
TAIL_DELAY_NAS_weight                int64
TAIL_DELAY_LATE_AIRCRAFT_weight      int64
dtype: object


In [ ]:
# ERROR- Unable to allocate 6. GiB for an array with shape (1617105, 441) and data type bool
#so to manage space convert 64 bit as 32 bit for each column variable(while taking whole data set)


In [ ]:
systematic_samplefinal=systematic_sample6.copy()
systematic_samplefinal[['MONTH','DAY_OF_MONTH','DAY_OF_WEEK']]=systematic_samplefinal[['MONTH','DAY_OF_MONTH','DAY_OF_WEEK']].astype(str)

systematic_samplefinal = systematic_samplefinal.astype({col: 'float16' for col in systematic_samplefinal.select_dtypes('float64').columns})
systematic_samplefinal = systematic_samplefinal.astype({col: 'int16' for col in systematic_samplefinal.select_dtypes('int64').columns})


### One Hot Encoding with get_dummy variables and drop first


In [ ]:
systematic_samplefinal = pd.get_dummies(systematic_samplefinal, drop_first=True)
systematic_samplefinal.dtypes

DEP_DEL15                     int16
TAIL_DELAY_weight             int16
TAIL_DELAY_CARRIER_weight     int16
TAIL_DELAY_SECURITY_weight    int16
TAIL_DELAY_WEATHER_weight     int16
                              ...  
ORIGIN_XNA                    uint8
ORIGIN_XWA                    uint8
ORIGIN_YAK                    uint8
ORIGIN_YKM                    uint8
ORIGIN_YUM                    uint8
Length: 440, dtype: object

In [ ]:
#systematic_samplefinal.head(5)

In [ ]:
systematic_samplefinal.shape

(215614, 440)

In [ ]:
y = systematic_samplefinal.DEP_DEL15 
 #Drop target and set the dependent values variables 
X = systematic_samplefinal.drop('DEP_DEL15', axis=1)
print(X.shape,y.shape)



(215614, 439) (215614,)


#### To avoid data leakage first do train test split and do pca, scaling etc..


In [ ]:
#Split into training and test sets
#Stratify will make sure your train and validation data are split based on output label frequencies based on train data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify = systematic_samplefinal['DEP_DEL15'])
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(161710, 439) (53904, 439) (161710,) (53904,)


In [ ]:
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
Rscaler = RobustScaler()
X_Train_scaled = Rscaler.fit_transform(X_train)
X_Test_scaled = Rscaler.transform(X_test)
#convert it to numpy arrays
#X_train=X_train.values
#X_test=X_test.values
#scaler = StandardScaler()
#X_Train_scaled = scaler.fit_transform(X_train)
#X_Test_scaled = scaler.transform(X_test)
#scaled_df = pd.DataFrame(scaled_df)
#X_Train = scale.fit_transform(X_Train)
#X_Test = scale.transform(X_Test)
#Rscaler = RobustScaler()
#X_train = Rscaler.fit_transform(X_train)


In [ ]:
#X_Train_scaled

# PCA - fit_transform on Train data set and transform on test data set 

In [ ]:
from sklearn.decomposition import PCA
# 95% of the variance is retained.
#pca = PCA(n_components=2) # can chose n num of components (439 columns transfered to 2 columns)
#to maintain 95% varience it neeed 94 principal component(columns)
pca = PCA(.85)
principalComponents_train = pca.fit_transform(X_Train_scaled)
principalComponents_test =pca.transform(X_Test_scaled)
print(principalComponents_train.shape,principalComponents_test.shape)

(161710, 52) (53904, 52)


In [ ]:
principalDf_train = pd.DataFrame(data = principalComponents_train)
#columns = ['principal component 1', 'principal component 2', 'principal component 3','principal component 4']
#principalDf_trainXY = pd.concat([principalDf_train, y_train], axis = 1)
#principalDf_train
principalDf_test = pd.DataFrame(data = principalComponents_test)


In [ ]:
pca.explained_variance_ratio_

array([0.22373413, 0.11389125, 0.04713816, 0.03272552, 0.03086583,
       0.03046338, 0.02337835, 0.02259988, 0.0219265 , 0.02082441,
       0.02049534, 0.01870929, 0.01832915, 0.01692398, 0.01405692,
       0.01074039, 0.00881231, 0.00839608, 0.00751728, 0.0070069 ,
       0.00638836, 0.00627699, 0.00562454, 0.0053422 , 0.00533163,
       0.00512741, 0.00494627, 0.00488491, 0.00486451, 0.00483396,
       0.00481371, 0.00478126, 0.00476648, 0.00475742, 0.00470628,
       0.00465695, 0.00462539, 0.00459828, 0.00457246, 0.00456213,
       0.00452441, 0.0044769 , 0.00447284, 0.00446951, 0.00441604,
       0.00440765, 0.00438365, 0.0043394 , 0.00429643, 0.00427537,
       0.00419188, 0.00400618])

In [ ]:
#y_train.value_counts()

SMOTE after PCA(applying SMOTE only for train dataset to avoid data leakage)- Trials to yields Higher accuracy

In [ ]:
oversample = SMOTE()
df_X_smt_PCA_train, df_Y_smt_PCA_train = oversample.fit_resample(principalDf_train, y_train)
counter = Counter(df_Y_smt_PCA_train)
print(counter)

Counter({0: 143845, 1: 143845})


# Logistic regression with Dataframe 1- SMOTE and PCA

In [ ]:
from sklearn.linear_model import LogisticRegression
model_LOGISTICS = LogisticRegression()
model_LOGISTICSfit = model_LOGISTICS.fit(df_X_smt_PCA_train, df_Y_smt_PCA_train)

In [ ]:
y_pred_test = model_LOGISTICSfit.predict(principalDf_test)
# Measuring Goodness of fit in Training data
print('R2 Value logistic:',metrics.r2_score(df_Y_smt_PCA_train, model_LOGISTICSfit.predict(df_X_smt_PCA_train)))
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))

R2 Value logistic: -0.5032986895616809
[[30610 17339]
 [ 2421  3534]]
              precision    recall  f1-score   support

           0       0.93      0.64      0.76     47949
           1       0.17      0.59      0.26      5955

    accuracy                           0.63     53904
   macro avg       0.55      0.62      0.51     53904
weighted avg       0.84      0.63      0.70     53904



## RANDOM FOREST - with DATAFRAME1- PCA and SMOTE

In [ ]:
model_Randomforest_smt= RandomForestClassifier(n_estimators= 10, criterion="entropy")  
model_Randomforest_smt = model_Randomforest_smt.fit(df_X_smt_PCA_train, df_Y_smt_PCA_train)
y_pred_test_RF = model_Randomforest_smt.predict(principalDf_test)
print('R2 Value RANDOM FOREST:',metrics.r2_score(df_Y_smt_PCA_train, model_Randomforest_smt.predict(df_X_smt_PCA_train)))
print(confusion_matrix(y_test, y_pred_test_RF))
print(classification_report(y_test, y_pred_test_RF))

R2 Value RANDOM FOREST: 0.9851784907365567
[[43533  4416]
 [ 4611  1344]]
              precision    recall  f1-score   support

           0       0.90      0.91      0.91     47949
           1       0.23      0.23      0.23      5955

    accuracy                           0.83     53904
   macro avg       0.57      0.57      0.57     53904
weighted avg       0.83      0.83      0.83     53904



# APPROACH2-Hyper parameter tunning in Random forest with Cross validation- Without smote and PCA

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean
# setting the class_weight argument on the RandomForestClassifier class.
#This argument takes a dictionary with a mapping of each class value (e.g. 0 and 1) to the weighting. 
#The argument value of ‘balanced‘ can be provided to automatically use the inverse weighting from the training dataset, giving focus to the minority class.
modelRF2 = RandomForestClassifier(n_estimators=10, class_weight='balanced')
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores_ypred = cross_val_score(modelRF2, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
#print(confusion_matrix(y, scores_ypred))
#print(classification_report(y, scores_ypred))
# summarize performance
print('Mean ROC AUC: %.3f' % mean(scores_ypred))

Mean ROC AUC: 0.626


# APPROACH 3- Hyper parameter tunning techniques-Randomized search CV and Grid Search CV

In [ ]:
#IF i give all parameters its taking an hour to run
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 2, stop = 10, num = 5)]
# Number of features to consider at every split
#max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
#max_depth = [int(x) for x in np.linspace(10, 100,5)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(random_grid)




{'n_estimators': [2, 4, 6, 8, 10], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'criterion': ['entropy', 'gini']}


In [ ]:
rf = RandomForestClassifier()
Skfold = StratifiedKFold(n_splits=5)

random_search = RandomizedSearchCV(rf, param_distributions=random_grid,
                                   n_iter=3, 
                                   cv = Skfold)
### fit the randomized model
random_search.fit(principalDf_train,y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_sp...
                     

In [ ]:
random_search.best_params_

{'criterion': 'gini',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 10}

In [ ]:
random_search

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_sp...
                     

In [ ]:
best_random_grid=random_search.best_estimator_


In [ ]:
from sklearn.metrics import accuracy_score
y_pred_RF=best_random_grid.predict(principalDf_test)
print(confusion_matrix(y_test,y_pred_RF))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred_RF)))
print("Classification report: {}".format(classification_report(y_test,y_pred_RF)))

[[47428   521]
 [ 5668   287]]
Accuracy Score 0.8851847729296527
Classification report:               precision    recall  f1-score   support

         0.0       0.89      0.99      0.94     47949
         1.0       0.36      0.05      0.08      5955

    accuracy                           0.89     53904
   macro avg       0.62      0.52      0.51     53904
weighted avg       0.83      0.89      0.84     53904



# APPROACH 4- class:~imblearn.pipeline.Pipeline` object (or make_pipeline helper function) working with transformers and resamplers.

# Pipeline flow- 
  * FinalDF->scaling(robust)-> train test split-> PCA->SMOTE-> models(KNN/ Random forest)feed pipeline in StratifiedKFold-> feed it to RandomizedSearchCV(hyper parameter tunning technique)


Running for more than 1.30hrs  

In [ ]:
from sklearn.model_selection import cross_val_predict

Rscaler = RobustScaler()
pca = PCA(.80)
smote = SMOTE(random_state=70)
knn = KNeighborsClassifier(n_neighbors=10)

pipeline = make_pipeline(Rscaler, pca,smote,knn)
#X_train, X_test, y_train, y_test
kf = StratifiedKFold(n_splits=5)
y_pred_KNN = cross_val_predict(pipeline,X, y, cv=kf)
conf_mat_KNN = confusion_matrix(y_test, y_pred_KNN)
print(conf_mat_KNN)
acc_Test_KNN= accuracy_score(y_test, y_pred_KNN)
print(acc_Test_KNN)
print(classification_report(y_test, y_pred_KNN))

KeyboardInterrupt: ignored

# FORMING NEW DATAFRAME

# APPROACH 5- To reduce column count and onhot encoding , lets do weightage calculation for all the categorical columns also.[ forming new DATAFRAME witn All numeric category Weightage]

### Day of week Vs DEP_DEL15 count

In [ ]:
systematic_sample6['DEP_DEL15'].value_counts()

0.0    191794
1.0     23820
Name: DEP_DEL15, dtype: int64

In [ ]:

systematic_sample6[['MONTH','DAY_OF_MONTH','DAY_OF_WEEK']]=systematic_sample6[['MONTH','DAY_OF_MONTH','DAY_OF_WEEK']].astype(int)
systematic_sample6[['DEP_DEL15']]=systematic_sample6[['DEP_DEL15']].astype(int)

systematic_sample6 = systematic_sample6.astype({col: 'float16' for col in systematic_sample6.select_dtypes('float64').columns})
systematic_sample6 = systematic_sample6.astype({col: 'int16' for col in systematic_sample6.select_dtypes('int64').columns})

systematic_sample6.dtypes

MONTH                               int16
DAY_OF_MONTH                        int16
DAY_OF_WEEK                         int16
OP_CARRIER                         object
ORIGIN                             object
DEP_DEL15                           int16
TAIL_DELAY_weight                   int16
TAIL_DELAY_CARRIER_weight           int16
TAIL_DELAY_SECURITY_weight          int16
TAIL_DELAY_WEATHER_weight           int16
TAIL_DELAY_NAS_weight               int16
TAIL_DELAY_LATE_AIRCRAFT_weight     int16
dtype: object

In [ ]:
cols=['DAY_OF_WEEK','TOTALFLIGHT_WEEKDAY','DEPDELAY_WEEKDAY','WEEK_DEP_DELAY_PERCENTAGE']
lst = []
for week in range(1,8):
  WEEK=week
  weekflighttot=systematic_sample6['DAY_OF_WEEK'][systematic_sample6.DAY_OF_WEEK==week].count()
  week_all=systematic_sample6[systematic_sample6['DAY_OF_WEEK']==week]
  weekdepdel=week_all['DEP_DEL15'][week_all['DEP_DEL15']==1].sum()
  percentweekdepdel=(weekdepdel/weekflighttot)*100
  lst.append([WEEK,weekflighttot,weekdepdel,percentweekdepdel])
WEEKWISE_DEP_DELAY_TABLE = pd.DataFrame(lst, columns=cols)
WEEKWISE_DEP_DELAY_TABLE['WEEK_DELAY_weight'] = pd.qcut(WEEKWISE_DEP_DELAY_TABLE['WEEK_DEP_DELAY_PERCENTAGE'], 7 , labels = False,duplicates='drop')
print(WEEKWISE_DEP_DELAY_TABLE)
systematic_sample7 = systematic_sample6.merge(WEEKWISE_DEP_DELAY_TABLE, left_on='DAY_OF_WEEK', right_on='DAY_OF_WEEK')
systematic_sample7.drop(columns=['TOTALFLIGHT_WEEKDAY'	,'DEPDELAY_WEEKDAY',	'WEEK_DEP_DELAY_PERCENTAGE'], inplace=True, axis=1)
#systematic_sample7

   DAY_OF_WEEK  ...  WEEK_DELAY_weight
0            1  ...                  5
1            2  ...                  1
2            3  ...                  0
3            4  ...                  2
4            5  ...                  3
5            6  ...                  4
6            7  ...                  6

[7 rows x 5 columns]


### Day of month Vs DEP_DEL15 count

In [ ]:
cols=['DAY_OF_MONTH','TOTAL_FLIGHTS','DAY_OF_MONTH_DELAYED_FLIGHTS','%DAYMONTH_DEP_DELAYED']
lst = []
for daymonth in range(1,32):
  MONTHday=daymonth
  monthdayflighttot=systematic_sample7['DAY_OF_MONTH'][systematic_sample7.DAY_OF_MONTH==daymonth].count()
  monthday_all=systematic_sample7[systematic_sample7['DAY_OF_MONTH']==daymonth]
  monthdaydepdel=monthday_all['DEP_DEL15'][monthday_all['DEP_DEL15']==1].sum()
  percentmonthdaydepdel=(monthdaydepdel/monthdayflighttot)*100
  lst.append([MONTHday,monthdayflighttot,monthdaydepdel,percentmonthdaydepdel])
MONTHDAYWISE_DEP_DELAY_TABLE = pd.DataFrame(lst, columns=cols)
MONTHDAYWISE_DEP_DELAY_TABLE['DAY_OF_MONTH_DELAY_weight'] = pd.qcut(MONTHDAYWISE_DEP_DELAY_TABLE['%DAYMONTH_DEP_DELAYED'], 31 , labels = False,duplicates='drop')
print(MONTHDAYWISE_DEP_DELAY_TABLE)
systematic_sample8 = systematic_sample7.merge(MONTHDAYWISE_DEP_DELAY_TABLE, left_on='DAY_OF_MONTH', right_on='DAY_OF_MONTH')
systematic_sample8.drop(columns=['TOTAL_FLIGHTS','DAY_OF_MONTH_DELAYED_FLIGHTS','%DAYMONTH_DEP_DELAYED'], inplace=True, axis=1)
#systematic_sample8

    DAY_OF_MONTH  ...  DAY_OF_MONTH_DELAY_weight
0              1  ...                         22
1              2  ...                         10
2              3  ...                         26
3              4  ...                         15
4              5  ...                          0
5              6  ...                          1
6              7  ...                          2
7              8  ...                          4
8              9  ...                          5
9             10  ...                         12
10            11  ...                         27
11            12  ...                         21
12            13  ...                         19
13            14  ...                         14
14            15  ...                         29
15            16  ...                         30
16            17  ...                         23
17            18  ...                         28
18            19  ...                         13
19            20  ..

### MONTH vs DEP_DEL15 count

In [ ]:
cols=['MONTH','TOTAL_FLIGHTS','MONTH_DELAYED_FLIGHTS','%_DEP_DELAYED']
lst = []
for monthvar in range(1,6):
  MONTH=monthvar
  monthflighttot=systematic_sample8['MONTH'][systematic_sample8.MONTH==monthvar].count()
  month_all=systematic_sample8[systematic_sample8['MONTH']==monthvar]
  monthdepdel=month_all['DEP_DEL15'][month_all['DEP_DEL15']==1].sum()
  percentmonthdepdel=(monthdepdel/monthflighttot)*100
  lst.append([MONTH,monthflighttot,monthdepdel,percentmonthdepdel])
MONTHWISE_DEP_DELAY_TABLE = pd.DataFrame(lst, columns=cols)
MONTHWISE_DEP_DELAY_TABLE['MONTH_DELAY_weight'] = pd.qcut(MONTHWISE_DEP_DELAY_TABLE['%_DEP_DELAYED'], 5, labels = False)
#MONTHWISE_DEP_DELAY_TABLE
systematic_sample9 = systematic_sample8.merge(MONTHWISE_DEP_DELAY_TABLE, left_on='MONTH', right_on='MONTH')
systematic_sample9.drop(columns=['TOTAL_FLIGHTS','MONTH_DELAYED_FLIGHTS','%_DEP_DELAYED'], inplace=True, axis=1)
#systematic_sample9

### ORIGIN vs DEP_DEL15

In [ ]:
origin_del = systematic_sample9[['ORIGIN','DEP_DEL15']].groupby('ORIGIN').sum().sort_values(by='DEP_DEL15',ascending=False).reset_index()
origin_del.rename({'DEP_DEL15':'ORIGIN_DELAY_COUNT'},inplace=True, axis=1)
origin_del['ORIGIN_weight'] = pd.qcut(origin_del['ORIGIN_DELAY_COUNT'], 35, labels = False,duplicates='drop')
print(origin_del)
systematic_sample10 = systematic_sample9.merge(origin_del, left_on='ORIGIN', right_on='ORIGIN')
systematic_sample10.drop(columns=['ORIGIN_DELAY_COUNT'], inplace=True, axis=1)
systematic_sample10

    ORIGIN  ORIGIN_DELAY_COUNT  ORIGIN_weight
0      DFW                1706             27
1      DEN                1411             27
2      ATL                1304             27
3      ORD                 789             27
4      MCO                 755             27
..     ...                 ...            ...
368    SMX                   0              0
369    HIB                   0              0
370    BQK                   0              0
371    BPT                   0              0
372    MVY                   0              0

[373 rows x 3 columns]


### OP_CARRIER_AIRLINES vs DEP_DEL15

In [ ]:
op_carrier_delay = systematic_sample10[['OP_CARRIER','DEP_DEL15']].groupby('OP_CARRIER').sum().sort_values(by='DEP_DEL15',ascending=False).reset_index()
op_carrier_delay.rename({'DEP_DEL15':'OP_CARRIER_DELAY_COUNT'},inplace=True, axis=1)
op_carrier_delay['OP_CARRIER_DELAY_COUNT_weight'] = pd.qcut(op_carrier_delay['OP_CARRIER_DELAY_COUNT'], 15, labels = False,duplicates='drop')
print(op_carrier_delay)
systematic_sample11 = systematic_sample10.merge(op_carrier_delay, left_on='OP_CARRIER', right_on='OP_CARRIER')
systematic_sample11.drop(columns=['OP_CARRIER_DELAY_COUNT'], inplace=True, axis=1)
#systematic_sample11
systematic_sample11.drop(columns=['MONTH',	'DAY_OF_MONTH',	'DAY_OF_WEEK',	'OP_CARRIER'	,'ORIGIN'], inplace=True, axis=1)


   OP_CARRIER  OP_CARRIER_DELAY_COUNT  OP_CARRIER_DELAY_COUNT_weight
0          WN                    5611                             14
1          AA                    2725                             14
2          OO                    2483                             13
3          DL                    2154                             12
4          UA                    1223                             12
5          B6                    1163                             11
6          MQ                    1009                             10
7          YX                     994                              9
8          NK                     944                              9
9          YV                     790                              8
10         G4                     706                              7
11         OH                     677                              7
12         F9                     643                              6
13         AS                     

New Dataframe 

In [ ]:
systematic_sample11

,DEP_DEL15,TAIL_DELAY_weight,TAIL_DELAY_CARRIER_weight,TAIL_DELAY_SECURITY_weight,TAIL_DELAY_WEATHER_weight,TAIL_DELAY_NAS_weight,TAIL_DELAY_LATE_AIRCRAFT_weight,WEEK_DELAY_weight,DAY_OF_MONTH_DELAY_weight,MONTH_DELAY_weight,ORIGIN_weight,OP_CARRIER_DELAY_COUNT_weight
0,0,3,58,0,0,15,0,6,23,0,27,12
1,0,3,82,0,0,14,34,6,23,0,27,12
2,0,2,39,0,0,10,0,6,23,0,27,12
3,1,4,57,0,0,31,23,6,12,0,27,12
4,0,3,45,0,8,19,8,6,24,0,27,12
...,...,...,...,...,...,...,...,...,...,...,...,...
215609,0,0,0,0,0,0,0,6,12,0,0,0
215610,0,0,0,0,0,0,0,0,1,0,0,0
215611,0,0,0,0,0,0,0,4,10,0,0,0
215612,0,0,0,0,0,0,0,3,4,0,0,0


In [ ]:
y11 = systematic_sample11.DEP_DEL15 
 #Drop target and set the dependent values variables 
X11 = systematic_sample11.drop('DEP_DEL15', axis=1)
print(X11.shape,y11.shape)

(215614, 11) (215614,)


## Trying with same pipeline- using Undersampling
* NearMiss-1: Majority class examples with minimum average distance to three closest minority class examples.
* NearMiss-2: Majority class examples with minimum average distance to three furthest minority class examples.
* NearMiss-3: Majority class examples with minimum distance to each minority class example.

### KNN

In [ ]:
from sklearn.model_selection import cross_val_predict
from imblearn.under_sampling import NearMiss
from sklearn.neighbors import KNeighborsClassifier
from imblearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import StratifiedKFold
Rscaler = RobustScaler()
pca = PCA(.95)
# define the undersampling method
undersample = NearMiss(version=3, n_neighbors_ver3=3)
knn = KNeighborsClassifier(n_neighbors=10)

pipeline = make_pipeline(Rscaler,pca,undersample,knn)
#X_train, X_test, y_train, y_test
kf = StratifiedKFold(n_splits=3)
y_pred_KNN = cross_val_predict(pipeline,X11, y11, cv=kf)
conf_mat_KNN = confusion_matrix(y11, y_pred_KNN)
print(conf_mat_KNN)
acc_Test_KNN= accuracy_score(y11, y_pred_KNN)
print(acc_Test_KNN)
print(classification_report(y11, y_pred_KNN))

[[105908  85886]
 [ 13162  10658]]
0.5406235216637139
              precision    recall  f1-score   support

           0       0.89      0.55      0.68    191794
           1       0.11      0.45      0.18     23820

    accuracy                           0.54    215614
   macro avg       0.50      0.50      0.43    215614
weighted avg       0.80      0.54      0.63    215614



### RANDOM FOREST

In [ ]:
from sklearn.model_selection import cross_val_predict
from imblearn.under_sampling import NearMiss
from sklearn.neighbors import KNeighborsClassifier
from imblearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import StratifiedKFold,cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

Rscaler = RobustScaler()
pca = PCA(.95)
# define the undersampling method
undersample = NearMiss(version=3, n_neighbors_ver3=3)
Randomforest_Usample= RandomForestClassifier(n_estimators= 10, criterion="entropy")  

pipeline = make_pipeline(Rscaler,pca,undersample,Randomforest_Usample)
#X_train, X_test, y_train, y_test
kf = StratifiedKFold(n_splits=5)
y_pred_RF_USample = cross_val_predict(pipeline,X11, y11, cv=kf)
print(y_pred_RF_USample)
conf_mat_RF_Usample = confusion_matrix(y11, y_pred_RF_USample)
print(conf_mat_RF_Usample)
acc_Test_RF_Usample= accuracy_score(y11, y_pred_RF_USample)
print(acc_Test_RF_Usample)
print(classification_report(y11, y_pred_RF_USample))

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

#kfold = model_selection.KFold(n_splits=10, random_state=42)
#model=RandomForestClassifier(n_estimators=50) '''

results =cross_validate(pipeline,X11, y11, cv=kf,
                                          scoring=scoring)

print('\n',results)

[1 1 1 ... 0 0 0]
[[94304 97490]
 [11386 12434]]
0.4950420659140872
              precision    recall  f1-score   support

           0       0.89      0.49      0.63    191794
           1       0.11      0.52      0.19     23820

    accuracy                           0.50    215614
   macro avg       0.50      0.51      0.41    215614
weighted avg       0.81      0.50      0.58    215614


 {'fit_time': array([5.85342216, 6.20873332, 8.34790158, 5.55002189, 5.50049591]), 'score_time': array([0.17283058, 0.17209458, 0.17419362, 0.18345523, 0.16780424]), 'test_accuracy': array([0.41959047, 0.49020244, 0.42708068, 0.50847576, 0.62184036]), 'test_precision': array([0.10337809, 0.10838716, 0.0962831 , 0.13892063, 0.12602864]), 'test_recall': array([0.55436608, 0.50020991, 0.49916037, 0.66351805, 0.40827036]), 'test_f1_score': array([0.17426017, 0.17816822, 0.16142828, 0.22974053, 0.19260286])}


### KNN with SMOTE oversampling Technique

In [ ]:
Rscaler = RobustScaler()
pca = PCA(.95)
# define the oversampling method

oversample = SMOTE()
#undersample = NearMiss(version=3, n_neighbors_ver3=3)
knn = KNeighborsClassifier(n_neighbors=10)

pipeline_smt = make_pipeline(Rscaler,pca,oversample,knn)
#X_train, X_test, y_train, y_test
kf = StratifiedKFold(n_splits=3)
y_pred_KNN_smt = cross_val_predict(pipeline_smt,X11, y11, cv=kf)
conf_mat_KNN_smt = confusion_matrix(y11, y_pred_KNN_smt)
print(conf_mat_KNN_smt)
acc_Test_KNN_smt= accuracy_score(y11, y_pred_KNN_smt)
print(acc_Test_KNN_smt)
print(classification_report(y11, y_pred_KNN_smt))

[[129080  62714]
 [ 14555   9265]]
0.641632732568386
              precision    recall  f1-score   support

           0       0.90      0.67      0.77    191794
           1       0.13      0.39      0.19     23820

    accuracy                           0.64    215614
   macro avg       0.51      0.53      0.48    215614
weighted avg       0.81      0.64      0.71    215614



### RANDOM FOREST WITH SMOTE

In [ ]:
Rscaler = RobustScaler()
pca = PCA(.95)
# define the oversampling method

oversample = SMOTE()
Randomforest_Usample= RandomForestClassifier(n_estimators= 10, criterion="entropy")  

pipeline_RFsmt = make_pipeline(Rscaler,pca,oversample,Randomforest_Usample)
#X_train, X_test, y_train, y_test
kf = StratifiedKFold(n_splits=5)
y_pred_RF_OSample = cross_val_predict(pipeline_RFsmt,X11, y11, cv=kf)
print(y_pred_RF_OSample)
conf_mat_RF_Osample = confusion_matrix(y11, y_pred_RF_OSample)
print(conf_mat_RF_Osample)
acc_Test_RF_Osample= accuracy_score(y11, y_pred_RF_OSample)
print(acc_Test_RF_Osample)
print(classification_report(y11, y_pred_RF_OSample))

[0 0 0 ... 0 0 0]
[[163234  28560]
 [ 19467   4353]]
0.777254723719239
              precision    recall  f1-score   support

           0       0.89      0.85      0.87    191794
           1       0.13      0.18      0.15     23820

    accuracy                           0.78    215614
   macro avg       0.51      0.52      0.51    215614
weighted avg       0.81      0.78      0.79    215614



## xgboost with PCA and Smote

In [ ]:
from xgboost import XGBClassifier


In [ ]:
# define model
#The scale_pos_weight value is used to scale the gradient for the positive class(1) in imbalance dataset.
XGBmodel = XGBClassifier(scale_pos_weight=100)
Rscaler = RobustScaler()
pca = PCA(.95)
# define the oversampling method

oversample = SMOTE()

pipeline_XGBsmt = make_pipeline(Rscaler,pca,oversample,XGBmodel)
#X_train, X_test, y_train, y_test
kf = StratifiedKFold(n_splits=5)
y_pred_XGB_OSample = cross_val_predict(pipeline_XGBsmt,X11, y11, cv=kf)
print(y_pred_XGB_OSample)
conf_mat_XGB_Osample = confusion_matrix(y11, y_pred_XGB_OSample)
print(conf_mat_XGB_Osample)
acc_Test_XGB_Osample= accuracy_score(y11, y_pred_XGB_OSample)
print(acc_Test_XGB_Osample)
print(classification_report(y11, y_pred_XGB_OSample))

[1 1 1 ... 0 0 0]
[[  1983 189811]
 [    92  23728]]
0.11924550353873126
              precision    recall  f1-score   support

           0       0.96      0.01      0.02    191794
           1       0.11      1.00      0.20     23820

    accuracy                           0.12    215614
   macro avg       0.53      0.50      0.11    215614
weighted avg       0.86      0.12      0.04    215614



### Try2- XGB without smote

In [ ]:
# define model
#The scale_pos_weight value is used to scale the gradient for the positive class(1) in imbalance dataset.
XGBmodel = XGBClassifier(scale_pos_weight=70)
#Rscaler = RobustScaler()
#pca = PCA(.95)
# define the oversampling method

#oversample = SMOTE()

pipeline_XGB= make_pipeline(XGBmodel)
#X_train, X_test, y_train, y_test
kf = StratifiedKFold(n_splits=5)
y_pred_XGB = cross_val_predict(pipeline_XGB,X11, y11, cv=kf)
print(y_pred_XGB)
conf_mat_XGB = confusion_matrix(y11, y_pred_XGB)
print(conf_mat_XGB)
acc_Test_XGB= accuracy_score(y11, y_pred_XGB)
print(acc_Test_XGB)
print(classification_report(y11, y_pred_XGB))

[1 1 1 ... 0 0 0]
[[  2058 189736]
 [   143  23677]]
0.11935681356498186
              precision    recall  f1-score   support

           0       0.94      0.01      0.02    191794
           1       0.11      0.99      0.20     23820

    accuracy                           0.12    215614
   macro avg       0.52      0.50      0.11    215614
weighted avg       0.84      0.12      0.04    215614



# Approach 6 - Hyper Parameter Tunning Randomised search CV for new Data frame

In [ ]:
X_train11, X_test11, y_train11, y_test11 = train_test_split(X11, y11, test_size=0.25, stratify = systematic_sample11['DEP_DEL15'])
print(X_train11.shape, X_test11.shape, y_train11.shape, y_test11.shape)

(161710, 11) (53904, 11) (161710,) (53904,)


In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 2, stop = 10, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100,5)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features':max_features,
               'max_depth' : max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(random_grid)

{'n_estimators': [2, 4, 6, 8, 10], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 32, 55, 77, 100], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'criterion': ['entropy', 'gini']}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestClassifier()
Skfold = StratifiedKFold(n_splits=5)

random_search = RandomizedSearchCV(rf, param_distributions=random_grid,
                                   n_iter=3, 
                                   cv = Skfold)
### fit the randomized model
random_search.fit(X_train11, y_train11)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_sp...
                     

In [ ]:
random_search.best_params_

{'criterion': 'entropy',
 'max_depth': 55,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 6}

In [ ]:
random_search

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_sp...
                     

In [ ]:
best_random_grid=random_search.best_estimator_


In [ ]:
from sklearn.metrics import accuracy_score
y_pred_RF11=best_random_grid.predict(X_test11)
print(confusion_matrix(y_test11,y_pred_RF11))
print("Accuracy Score {}".format(accuracy_score(y_test11,y_pred_RF11)))
print("Classification report: {}".format(classification_report(y_test11,y_pred_RF11)))

[[47340   609]
 [ 5708   247]]
Accuracy Score 0.8828101810626299
Classification report:               precision    recall  f1-score   support

           0       0.89      0.99      0.94     47949
           1       0.29      0.04      0.07      5955

    accuracy                           0.88     53904
   macro avg       0.59      0.51      0.50     53904
weighted avg       0.83      0.88      0.84     53904

